Initial attempt at saving the image from ChromeDino so that we can then try and make use of the game environment

In [8]:
import re
import time
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm, tnrange
import numpy as np
from collections import defaultdict,Counter

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

import pandas as pd
import base64

from matplotlib import pyplot as plt, rcParams
%matplotlib inline
rcParams["figure.figsize"] = (16,12)    # Use this line to increase your figure size (optional).

Stack overflow link talking about saving images in canvas: https://stackoverflow.com/questions/44485616/web-scraping-image-inside-canvas

# Option 1
Selenium is capable of taking a screenshot and saving it, here's a working example of it:

In [4]:
url = "https://chromedino.com/"
browser = webdriver.Chrome()
try:
    browser.get(url)
    browser.save_screenshot('homepage.png')
except Exception as e:
    print("Errored out at step 1", e)
finally:
    browser.close() #make sure we close it when we are finished.

# Option 2
Force Selenium to execute JavaScript to just pull the information out of the canvas tag.

In [11]:
js_call = "return document.getElementsByClassName('runner-canvas')[0].toDataURL('image/png').substring(21);"
url = "https://chromedino.com/"
browser = webdriver.Chrome()
try:
    browser.get(url)

    body = browser.find_element_by_id('t')
    body.send_keys(Keys.ARROW_UP)
    time.sleep(5)
    
    b64_image = browser.execute_script(js_call)
    image = base64.b64decode(b64_image)
    output = open("game_board2.png", 'wb')
    output.write(image)
    output.close()
except Exception as e:
    print("Errored out at step 1", e)
finally:
    browser.close() #make sure we close it when we are finished.